# Master Discogs Database

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap

### Master DB code
from masterdb import masterdb
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

In [ ]:
mdb = myMusicDBMap(debug=True)

In [ ]:
maindb = mainDB(mdb=mdb, create=True, debug=True)
print("\n\n{0}\n".format("Full DB"))
#maindb.setDBFull("AllMusic")
#maindb.setDBFull("MusicBrainz")
#maindb.setDBFull("Discogs")
maindb.setDBFull("LastFM")
#maindb.setDBFull()
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

In [ ]:
maindb = mainDB(mdb=mdb, create=False, debug=True)

In [ ]:
maindb.setDBFull()

# Metadata

In [ ]:
%load_ext autoreload
%autoreload

from multiprocessing import Pool
from mainDB import mainDB
import time
mainDB = mainDB()
dbdata = mainDB.dbdata

def parseArtistsAM(modVal, force=False, doExtra=False):
    dbdata["AllMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=True, doExtra=False):
    dbdata["Discogs"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    dbdata["MusicBrainz"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsMB.parseArtistModValFiles(modVal, force=force)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)
    
def parseArtistsDP(modVal, force=False, doExtra=False):
    dbdata['DatPiff']['Artists'].parseArtistFiles()
    #artsDP.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    dbdata["RateYourMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    dbdata["LastFM"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsLM.parseArtistModValFiles(modVal, force=force)

def parseArtistsRC(modVal, force=False, doExtra=False):
    dbdata["RockCorner"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRC.parseArtistModValFiles(modVal, force=force)

def parseArtistsCL(modVal, force=False, doExtra=False):
    dbdata["CDandLP"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsCL.parseArtistModValFiles(modVal, force=force)

def parseArtistsMS(modVal, force=False, doExtra=False):
    artsMS.parseArtistFiles(force=force)

def parseArtistsMT(modVal, force=False, doExtra=False):
    artsMT.parseArtistModValFiles(modVal, force=force)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        dbdata["RateYourMusic"]["Artists"].parseDownloadedFiles()
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    elif db == "CDandLP":
        result = pool.map_async(parseArtistsCL, range(100))
        #result = pool.map_async(parseArtistsCL, range(56,72))
        #result = pool.map_async(parseArtistsCL, [55,25,26])
    elif db == "MusicStack":
        result = pool.map_async(parseArtistsMS, [None])
    elif db == "MetalStorm":
        result = pool.map_async(parseArtistsMT, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

In [ ]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
#parseArtistsParallel(db="MusicBrainz", nProcs=3)
parseArtistsParallel(db="Discogs", nProcs=4)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
#parseArtistsParallel(db="LastFM", nProcs=3)
#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['LastFM']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)